<a href="https://colab.research.google.com/github/MunYeong0723/NLP_mini-project/blob/master/Surprise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [321]:
# !pip install scikit-surprise

In [322]:
import pandas as pd
from ast import literal_eval
from surprise import SVD, Dataset, accuracy, Reader
from surprise.model_selection import train_test_split

In [323]:
pd.set_option('display.max_columns', 6)
pd.set_option('display.width', 300)

In [324]:
movie_list = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP_mini-project/data/list_movie.csv')
movie_list.columns = ['movie_code', 'title', 'year', 'genres', 'ratings_avg']
movie_list

,movie_code,title,year,genres,ratings_avg
0,md6B3Ad,대부 일대기,1992,"['드라마', '범죄']",4.4
1,mNdKBkO,쇼생크 탈출,1994,['드라마'],4.4
2,mV539wO,인생은 아름다워,1997,"['드라마', '코미디']",4.4
3,mW4LYBo,하이큐!! 콘셉트의 싸움,2017,"['애니메이션', '스포츠']",4.4
4,m5r0pVO,12인의 노한 사람들,1997,"['드라마', '범죄']",4.2
...,...,...,...,...,...
292,mWvDGZd,써니,2011,"['드라마', '코미디']",4.0
293,m5338w5,미녀와 야수,1991,"['애니메이션', '가족', '판타지', '로맨스', '키즈']",4.0
294,m5XMJ1W,노인을 위한 나라는 없다,2007,"['범죄', '스릴러', '드라마']",4.0
295,mVWzLNd,내가 널 사랑할 수 없는 10가지 이유,1999,"['로맨틱코미디', '틴에이저']",4.0


In [325]:
user_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP_mini-project/data/user_data.csv')

user_data.fillna("[]", inplace=True)

user_data['rt_movie'] = user_data['rt_movie'].apply(literal_eval)
user_data['wish_movie'] = user_data['wish_movie'].apply(literal_eval)

user_data

,user_code,rt_movie,wish_movie
0,6ADvGQwymxzZl,"[{'movie_code': 'mNdKBkO', 'movie_title': '쇼생크...","[{'code': 'mjO0zL5', 'title': '대부', 'year': 19..."
1,aJYMKqmk5loDW,"[{'movie_code': 'm5Q9Zg8', 'movie_title': '조커'...","[{'code': 'mOAnkZd', 'title': '그들만의 세상', 'year..."
2,87Gv7Z6QVqE6o,"[{'movie_code': 'm5Q9ZMR', 'movie_title': '터미네...","[{'code': 'mdRJybd', 'title': '더 텐 오클락 피플', 'y..."
3,36lvXKdAPqXdn,"[{'movie_code': 'mOPV7VD', 'movie_title': '인비저...","[{'code': 'mOlpNeO', 'title': '아바타 3', 'year':..."
4,17ovVmrwBqzyn,"[{'movie_code': 'mWvkk73', 'movie_title': '리차드...","[{'code': 'mOAZkmV', 'title': '벌집의 아이들', 'year..."
...,...,...,...
253,17ovV7K4R5zyn,"[{'movie_code': 'm5Q9Zg8', 'movie_title': '조커'...","[{'code': 'mdRLwyg', 'title': '코멘트 유콩 디플라카 레스 ..."
254,j4PxOEBKdvp0Q,"[{'movie_code': 'mOPV7VD', 'movie_title': '인비저...","[{'code': 'm5nXQq9', 'title': '신림', 'year': 20..."
255,2mwvgNnAOqMa7,"[{'movie_code': 'mNdKBkO', 'movie_title': '쇼생크...",[]
256,VRZv4n39Pxr6y,"[{'movie_code': 'mOPV7VD', 'movie_title': '인비저...","[{'code': 'mOAZ6VE', 'title': '컨페티 하베스트', 'yea..."


In [326]:
user_movie_rating_origin = pd.DataFrame(columns=['user_code', 'movie_code', 'movie_rating'])
user_movie_wish_origin = pd.DataFrame(columns=['user_code', 'movie_code'])

for i in range(len(user_data)):
    user = user_data.iloc[i]
    user_code = user['user_code']
    for mv in user['rt_movie']:
        user_movie_rating_origin = user_movie_rating_origin.append({'user_code' : user_code,
                                                                    'movie_code' : mv['movie_code'],
                                                                    'movie_rating': mv['movie_rating']
                                                                }, ignore_index=True)

    for wish in user['wish_movie']:
        user_movie_wish_origin = user_movie_wish_origin.append({'user_code' : user_code,
                                                                'movie_code' : wish['code']}, ignore_index=True)


print(user_movie_rating_origin)
print(user_movie_wish_origin)

          user_code movie_code  movie_rating
0     6ADvGQwymxzZl    mNdKBkO           4.0
1     6ADvGQwymxzZl    mV539wO           4.0
2     6ADvGQwymxzZl    mW4L2XW           3.0
3     6ADvGQwymxzZl    mJOVXR5           3.0
4     6ADvGQwymxzZl    mJ5x8x5           5.0
...             ...        ...           ...
2575  2mwvgZAyQqMa7    m5Q9ZMR           2.0
2576  2mwvgZAyQqMa7    mNdKBkO           5.0
2577  2mwvgZAyQqMa7    mV539wO           4.5
2578  2mwvgZAyQqMa7    mW4L2XW           5.0
2579  2mwvgZAyQqMa7    mJOVXR5           5.0

[2580 rows x 3 columns]
          user_code movie_code
0     6ADvGQwymxzZl    mjO0zL5
1     6ADvGQwymxzZl    mWy4nk5
2     6ADvGQwymxzZl    mO2MoxW
3     6ADvGQwymxzZl    mWvDoed
4     6ADvGQwymxzZl    mO1VNVd
...             ...        ...
2238  2mwvgZAyQqMa7    md7nK2O
2239  2mwvgZAyQqMa7    mO1D8PW
2240  2mwvgZAyQqMa7    mWyaxex
2241  2mwvgZAyQqMa7    m85XM3W
2242  2mwvgZAyQqMa7    m5DPDV6

[2243 rows x 2 columns]


In [327]:
# 정수 인코딩
movie_code = movie_list['movie_code'].unique().tolist()
movie_to_index = {x : i for i,x in enumerate(movie_code)}
index_to_movie = {i : x for i,x in enumerate(movie_code)}

user_code = pd.merge(user_movie_rating_origin['user_code'], user_movie_wish_origin['user_code'], on='user_code', how='outer')
user_code = user_code['user_code'].unique().tolist()
user_to_index = {x : i for i,x in enumerate(user_code)}
index_to_user = {i : x for i,x in enumerate(user_code)}

print(user_to_index)
print(index_to_user)

{'6ADvGQwymxzZl': 0, 'aJYMKqmk5loDW': 1, '87Gv7Z6QVqE6o': 2, '36lvXKdAPqXdn': 3, '17ovVmrwBqzyn': 4, 'ZBm5Ry4Wyvd46': 5, '8nPvyYQRPPvYo': 6, 'yKZx3Q9PEv4dJ': 7, 'DgwxAeQYNxrMj': 8, 'a9L5PwPPOvwg8': 9, '3BnvwnOBZqPAY': 10, 'OkexJVnVgxdbw': 11, 'JgkqlbMKD5X0b': 12, '87Gv7zgkmqE6o': 13, 'Mr95n1pbDvZPG': 14, 'RE952D72gqQ72': 15, '6ADvGQdzgxzZl': 16, 'Mr95nbQ175ZPG': 17, '6ADvGVWKKqzZl': 18, 'OkexJLenEvdbw': 19, '87Gv7Jy4LqE6o': 20, '17ovVQOL9qzyn': 21, '487Gv7rKJqE6o': 22, 'YMKqmppRrxloD': 23, 'YMKqmpDOyxloD': 24, 'D4WLxZjyrxroA': 25, 'JZ4vBbeMYvRXO': 26, 'R6OvKLlVW5N8V': 27, 'RE952bLDE5Q72': 28, 'jae5WMzyRq1P3': 29, 'Pznx9Kmk2Zv76': 30, 'djaxbLAoPvLw8': 31, 'YMr95nBkd5ZPG': 32, 'a9L5Pr4WOqwg8': 33, 'WRQxDDl6Pxdl9': 34, '6ADvGBykWqzZl': 35, '6ewxaBeAGqQ1m': 36, 'zNM5NkgYlx26j': 37, 'zNM5N0XoGq26j': 38, '6ADvG6gBdqzZl': 39, 'YMKqm2p7lvloD': 40, '36lvXPjGovXdn': 41, 'nkPvrb8wrvgar': 42, 'R6OvKeOr9xN8V': 43, '6ewxa3lLkvQ1m': 44, '36lvX1YAJ5Xdn': 45, 'K6ExjB1wwxX1O': 46, 'pnb4xkpAmqOAz': 47, '

In [328]:
movie_list['movie_code'] = movie_list['movie_code'].map(movie_to_index)

user_movie_rating = user_movie_rating_origin.copy()
user_movie_wish = user_movie_wish_origin.copy()

user_movie_rating['user_code'] = user_movie_rating['user_code'].map(user_to_index)
user_movie_rating['movie_code'] = user_movie_rating['movie_code'].map(movie_to_index)

user_movie_wish['user_code'] = user_movie_wish['user_code'].map(user_to_index)
user_movie_wish['movie_code'] = user_movie_wish['movie_code'].map(movie_to_index)

In [329]:
print(movie_list)
print(user_movie_rating)
print(user_movie_wish)

     movie_code                  title  year                               genres  ratings_avg
0             0                 대부 일대기  1992                        ['드라마', '범죄']          4.4
1             1                 쇼생크 탈출  1994                              ['드라마']          4.4
2             2               인생은 아름다워  1997                       ['드라마', '코미디']          4.4
3             3          하이큐!! 콘셉트의 싸움  2017                     ['애니메이션', '스포츠']          4.4
4             4            12인의 노한 사람들  1997                        ['드라마', '범죄']          4.2
..          ...                    ...   ...                                  ...          ...
292         292                     써니  2011                       ['드라마', '코미디']          4.0
293         293                 미녀와 야수  1991  ['애니메이션', '가족', '판타지', '로맨스', '키즈']          4.0
294         294          노인을 위한 나라는 없다  2007                 ['범죄', '스릴러', '드라마']          4.0
295         295  내가 널 사랑할 수 없는 10가지 이유  1999      

In [330]:
# 결측치 제거 -> movie_list에 없는 movie일 때
# list에 있는 영화만을 가지고 train하기 위해 일단 list에 없는 영화에 대한 평가 & wish 는 제거
user_movie_rating.dropna(axis=0, inplace=True)
user_movie_wish.dropna(axis=0, inplace=True)

print(user_movie_rating)
print(user_movie_wish)

      user_code  movie_code  movie_rating
0             0         1.0           4.0
1             0         2.0           4.0
2             0         5.0           3.0
3             0         7.0           3.0
4             0         8.0           5.0
...         ...         ...           ...
2574        257         0.0           4.5
2576        257         1.0           5.0
2577        257         2.0           4.5
2578        257         5.0           5.0
2579        257         7.0           5.0

[1761 rows x 3 columns]
      user_code  movie_code
0             0        16.0
2             0        94.0
5             0        79.0
6             0       205.0
9             0       191.0
...         ...         ...
2235        257        26.0
2236        257        50.0
2238        257        40.0
2239        257        34.0
2241        257        52.0

[901 rows x 2 columns]


In [331]:
repre_rating = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP_mini-project/data/rating_data_pre.csv')
repre_wish = pd.read_csv('/content/drive/My Drive/Colab Notebooks/NLP_mini-project/data/wish_data_pre.csv')

# title -> movie_code
title_to_code = movie_list[['title', 'movie_code']].set_index("title").T.to_dict('index')
title_to_code = title_to_code['movie_code']

# data preprocessing
repre_rating['movie_code'] = repre_rating['제목'].map(title_to_code)
repre_rating = repre_rating.dropna(axis=0)
repre_rating.columns = ['title', 'genres', 'total_avg', 'movie_rating', 'movie_code']

repre_wish['movie_code'] = repre_wish['제목'].map(title_to_code)
repre_wish = repre_wish.dropna(axis=0)
repre_wish.columns = ['title', 'genres', 'total_avg', 'movie_code']

repre_rating['user_code'] = [258 for _ in range(len(repre_rating))]
repre_wish['user_code'] = [258 for _ in range(len(repre_wish))]

repre_rating = repre_rating[['user_code', 'movie_code', 'movie_rating']]
repre_wish = repre_wish[['user_code', 'movie_code']]

# 중복 제거
repre_rating = repre_rating.drop_duplicates(['movie_code'])
repre_wish = repre_wish.drop_duplicates(['movie_code'])

# user_movie_rating 에 합치기
user_movie_rating = user_movie_rating.append(repre_rating)
user_movie_wish = user_movie_wish.append(repre_wish)

display(user_movie_rating)
display(user_movie_wish)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """


,user_code,movie_code,movie_rating
0,0,1.0,4.0
1,0,2.0,4.0
2,0,5.0,3.0
3,0,7.0,3.0
4,0,8.0,5.0
...,...,...,...
5028,258,95.0,4.0
5031,258,243.0,3.5
5044,258,227.0,3.5
5056,258,282.0,3.5


,user_code,movie_code
0,0,16.0
2,0,94.0
5,0,79.0
6,0,205.0
9,0,191.0
...,...,...
4121,258,192.0
4486,258,62.0
4676,258,189.0
4878,258,234.0


In [332]:
user_movie_rating['movie_code'] = user_movie_rating['movie_code'].apply(int)
user_movie_wish['movie_code'] = user_movie_wish['movie_code'].apply(int)

print(user_movie_rating)
print(user_movie_wish)

      user_code  movie_code  movie_rating
0             0           1           4.0
1             0           2           4.0
2             0           5           3.0
3             0           7           3.0
4             0           8           5.0
...         ...         ...           ...
5028        258          95           4.0
5031        258         243           3.5
5044        258         227           3.5
5056        258         282           3.5
5090        258          47           4.0

[1961 rows x 3 columns]
      user_code  movie_code
0             0          16
2             0          94
5             0          79
6             0         205
9             0         191
...         ...         ...
4121        258         192
4486        258          62
4676        258         189
4878        258         234
5259        258         274

[1142 rows x 2 columns]


In [333]:
# shuffle
user_movie_rating = user_movie_rating.sample(frac=1).reset_index()
print(user_movie_rating)

reader = Reader(rating_scale=(0.0, 5.0))
data = Dataset.load_from_df(df=user_movie_rating[['user_code', 'movie_code', 'movie_rating']], reader=reader)
print(data)

      index  user_code  movie_code  movie_rating
0       808         80          13           4.0
1       917         91           1           4.5
2      2489        248          11           2.0
3       179         17          10           4.5
4       298         29           8           4.0
...     ...        ...         ...           ...
1956    999         99          14           4.5
1957   2074        207           8           5.0
1958    595         59          10           4.0
1959   2468        246          54           4.0
1960   1613        161          12           3.5

[1961 rows x 4 columns]


In [334]:
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import *

algo_str = "SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering"
algo = [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(),KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]

benchmark = []
# iterate all algorithms
for i in range(len(algo)) :
    results = cross_validate(algo[i], data, measures=['RMSE', 'MAE'], cv=5, verbose=False)
    # print(results)
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([algo_str.split(", ")[i]], index=["Algorithm"]))
    benchmark.append(tmp)

pd.DataFrame(benchmark).set_index("Algorithm").sort_values(["test_rmse", "test_mae"])

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Don

,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
SVDpp,0.610572,0.472452,0.537123,0.007439
KNNBaseline,0.624880,0.476404,0.006228,0.057744
KNNWithMeans,0.629553,0.484378,0.006701,0.051134
SlopeOne,0.629799,0.474899,0.004721,0.003774
KNNWithZScore,0.631813,0.485909,0.014258,0.054273
SVD,0.635463,0.492869,0.094384,0.002839
BaselineOnly,0.640081,0.498722,0.001973,0.001462
KNNBasic,0.660821,0.507006,0.005150,0.047413
CoClustering,0.679638,0.519269,0.052217,0.002151


In [335]:
param_grid = {
    'n_epochs':[5,10,50,100],
    'n_factors' : [10,20,50,100]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5)
gs.fit(data)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.6085065730575304
{'n_epochs': 50, 'n_factors': 10}


#### fit & predict

In [336]:
user_id = 258
print(user_movie_rating['user_code'].unique())

[ 80  91 248  17  29 170  44 168  71 144 199 258  97  89  72  59 200  11
 140 102 234 155 151  45 159 240 150  23 230  82   8 217  87 246 237 146
 171  70 241 158 181 179  22  32  28   0 224  12 190 127 197 112  26  62
  98 138 152  10 141 209 256 249 185 207 117 189 114 119 250 165 226  43
 109  68  39 123  20 183 205  55 223 233   1 186 132  34  92  84  51 104
 162 221 193 175  35 219 173 255 231 251  30  13  75  99  38 105  63  77
 227   5 116 133  95 236 213 169 160  65  96  36 192  57 254  81 153  53
 106 177 218 157 122 198  42  46 245 130 215 172  24 253 126  40 210  27
 174 103 142 129  88  90 203  47 137 107 220 182  83 196  18 110  67 149
 113 135 147 145 202   6 154  31  37 204  60  79 100 238 225 118 176 184
 206 164 257  58 187 180  16 166  69 235   9 244 191 156   3  76  48  93
 195  64 247 232 194 120 211 216  50 161 188   2 115 125  49  19 239 131
  21 222  66  25  15  41  85 228 201  14 121 163 212   4 148 111   7  52
  33 229 243 101 167  94 139 136  56 242  61 208 21

In [337]:
algo = gs.best_estimator['rmse']
algo.fit(data.build_full_trainset())

pred = algo.predict(str(user_id), '200')
print(pred)

user: 258        item: 200        r_ui = None   est = 4.17   {'was_impossible': False}


In [338]:
preds = pd.DataFrame(columns=['pred_est', 'movie_code'])
for i in movie_list['movie_code'].unique() :
    pred = algo.predict(str(user_id), i)
    # preds.append((pred.est, pred.iid))

    preds = preds.append({
        'pred_est' : pred.est,
        'movie_code' : pred.iid
    }, ignore_index=True)

preds['movie_code'] = preds['movie_code'].apply(int)
preds = preds.sort_values(by='pred_est', ascending=False)
preds

,pred_est,movie_code
28,4.553255,28
16,4.529723,16
14,4.434716,14
7,4.412449,7
129,4.384048,129
...,...,...
17,3.856047,17
20,3.795236,20
107,3.794084,107
27,3.722853,27


In [339]:
movie_list[movie_list['movie_code'] == 28]

,movie_code,title,year,genres,ratings_avg
28,28,매트릭스,1999,"['판타지', 'SF', '액션', '모험']",4.2


### 높게 평가할 영화 + 보고 싶은 영화

In [340]:
user_movie_wish['user_code'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   9,  11,  13,  15,  16,
        18,  19,  21,  22,  23,  24,  26,  27,  28,  29,  30,  34,  35,
        39,  40,  41,  42,  43,  44,  46,  47,  48,  50,  51,  55,  56,
        57,  58,  62,  63,  64,  65,  66,  67,  68,  69,  70,  72,  75,
        76,  77,  79,  80,  81,  83,  84,  87,  88,  91,  92,  93,  95,
        97,  98,  99, 101, 102, 107, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 124, 126, 127, 129, 130, 132, 133, 134,
       137, 138, 139, 140, 141, 145, 146, 147, 148, 150, 151, 152, 154,
       155, 157, 158, 159, 160, 161, 162, 163, 166, 168, 169, 170, 171,
       173, 174, 175, 176, 177, 179, 180, 181, 183, 184, 185, 186, 189,
       191, 192, 194, 195, 196, 197, 199, 200, 201, 202, 203, 205, 207,
       209, 211, 212, 213, 214, 216, 217, 219, 220, 221, 223, 224, 226,
       227, 231, 232, 233, 235, 236, 237, 241, 243, 244, 245, 246, 247,
       249, 250, 252, 254, 256, 257, 258])

In [341]:
user_wish = user_movie_wish[user_movie_wish['user_code'] == user_id]

top_10 = pd.merge(user_wish, preds, on='movie_code', how='right').iloc[:10]
top_10 = top_10.sort_values(by='pred_est', ascending=False)
top_10

,user_code,movie_code,pred_est
7,258.0,87,4.224635
2,258.0,272,4.173023
0,258.0,215,4.171851
4,258.0,101,4.171851
6,258.0,59,4.171851
8,258.0,197,4.171851
9,258.0,113,4.171851
1,258.0,243,4.110737
5,258.0,266,4.109467
3,258.0,103,4.091954


In [342]:
pd.merge(top_10['movie_code'], movie_list, on='movie_code', how='left')

,movie_code,title,year,genres,ratings_avg
0,87,타인의 삶,2006,"['스릴러', '드라마']",4.0
1,272,1917,2019,"['드라마', '전쟁']",4.2
2,215,디지몬 어드벤처 1기: 운명적 만남,1999,"['애니메이션', '가족', '액션', '모험', '판타지', 'SF', '스릴러']",4.1
3,101,루스 베이더 긴즈버그: 나는 반대한다,2018,['다큐멘터리'],4.2
4,59,우먼 인 할리우드,2018,['다큐멘터리'],4.1
5,197,셜록 2세,1924,"['코미디', '가족']",4.1
6,113,카메라맨,1928,"['로맨스', '코미디', '가족']",4.1
7,243,더 폴: 오디어스와 환상의 문,2006,"['모험', '드라마', '판타지']",4.0
8,266,코미디의 왕,1983,"['드라마', '범죄', '코미디']",4.0
9,103,포드 V 페라리,2019,"['액션', '드라마']",4.0
